In this notebook, I run one step for TwoStreamControlModel to get intermediate outputs which I can then compare to my local run

In [1]:
!apt-get update -qq
!apt-get install -y libgl1-mesa-glx -qq

debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libelf1:amd64.
(Reading database ... 30662 files and directories currently installed.)
Preparing to unpack .../00-libelf1_0.176-1.1ubuntu0.1_amd64.deb ...
Unpacking libelf1:amd64 (0.176-1.1ubuntu0.1) ...
Selecting previously unselected package libdrm-common.
Preparing to unpack .../01-libdrm-common_2.4.107-8ubuntu1~20.04.2_all.deb ...
Unpacking libdrm-common (2.4.107-8ubuntu1~20.04.2) ...
Selecting previously unselected package libdrm2:amd64.
Preparing to unpack .../02-libdrm2_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
Unpacking libdrm2:amd64 (2.4.107-8ubuntu1~20.04.2) ...
Selecting previously unselected package libdrm-amdgpu1:amd64.
Preparing to unpack .../03-libdrm-amdgpu1_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
Unpacking libdrm-amdgpu1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
Selecting previously unselected package libpciaccess0:amd64.
Preparing to unpack .../04-libpciaccess0_0.1

In [7]:
!git clone https://github.com/vislearn/ControlNet-XS -qq

fatal: destination path 'ControlNet-XS' already exists and is not an empty directory.


In [8]:
!rm -rf diffusers
!git clone https://github.com/UmerHA/diffusers.git --branch controlnet-xs -qq

In [9]:
!pip install -r ControlNet-XS/requirements/pt2.txt -qq
!pip install -e ControlNet-XS -qq

In [10]:
!pip install -Uqq transformers
!pip install diffusers -e diffusers -qq

In [11]:
assert False

AssertionError: 

___

Restart kernel here, so newly installed packages are available

___

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # needed to make torch deterministic

In [2]:
import scripts.control_utils as cu
import torch
from PIL import Image

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [3]:
torch.use_deterministic_algorithms(True)

In [4]:
path_to_config = 'ControlNet-XS/configs/inference/sdxl/sdxl_encD_canny_48m.yaml'

If this results in the kernel crashing, I'm using too much GPU memory elsewhere. Shut down every other kernel and try again.

In [ ]:
model = cu.create_model(path_to_config).to('cuda')

Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 320, out-chn: 320, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 640, out-chn: 640, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing


In [6]:
time_linear_in_first_resnet = model.model.diffusion_model.input_blocks[1][0].emb_layers[1]

In [7]:
time_linear_in_first_resnet

Linear(in_features=1280, out_features=320, bias=True)

In [8]:
torch.save(time_linear_in_first_resnet.state_dict(), 'time_linear_in_first_resnet.pth')

In [10]:
size = 768
num_samples=1
prompt='cinematic, shoe in the streets, made from meat, photorealistic shoe, highly detailed'
n_prompt='lowres, bad anatomy, worst quality, low quality'
def get_canny_edges():
    image_path = 'input_images/shoe.png' # chosen to fit size above
    image = cu.get_image(image_path, size=size)
    edges = cu.get_canny_edges(image, low_th=100, high_th=250)
    return edges
edges = get_canny_edges()

guidance=edges
ddim_steps=10
num_samples=num_samples
shape=[4, size // 8, size // 8]
control_scale=0.95

scale=9.5,
eta=0.5,
idx=None,

ds = {
    'hint': guidance,
    'crop_coords_top_left': torch.tensor([0, 0]),
    'original_size_as_tuple': torch.tensor(guidance.shape[-2:]),
    'target_size_as_tuple': torch.tensor(guidance.shape[-2:]),
}

model.sampler.num_steps = ddim_steps
model.sampler.s_noise = eta
scale_schedule = lambda scale, sigma: scale  # independent of step
from functools import partial
model.sampler.guider.scale_schedule = partial(scale_schedule, scale)

if float(control_scale) != 1.0:
    model.model.scale_list *= control_scale
    print(f'[CONTROL CORRECTION OF {type(model).__name__} SCALED WITH {control_scale}]')

[CONTROL CORRECTION OF ControlledDiffusionEngine SCALED WITH 0.95]


In [11]:
import torchvision.transforms as tt
import numpy as np
import copy

control = torch.stack([tt.ToTensor()(ds['hint'][..., None].repeat(3, 2))] * num_samples).float().to('cuda')
sampling_kwargs = {'hint': control}

def get_batch(ds_instance, num_samples):
    batch = dict()
    for key in ds_instance:
        instance = ds_instance[key]
        if isinstance(instance, str):
            batch[key] = [instance] * num_samples
        elif isinstance(instance, np.ndarray):
            batch[key] = instance[None, ...].repeat(num_samples, 0)
        elif isinstance(instance, torch.Tensor):
            batch[key] = instance[None, ...].repeat(num_samples, *[1] * len(instance.shape))

    return batch

batch = get_batch(ds_instance=ds, num_samples=num_samples)
batch['caption'] = [prompt or ds['caption']] * num_samples

for k in batch:
    if isinstance(batch[k], torch.Tensor):
        batch[k] = batch[k].to('cuda')

batch_uc = copy.deepcopy(batch)
batch_uc['caption'] = [n_prompt] * num_samples

In [12]:
batch

{'hint': array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]),
 'crop_coords_top_left': tensor([[0, 0]], device='cuda:0'),
 'original_size_as_tuple': tensor([[768, 768]], device='cuda:0'),
 'target_size_as_tuple': tensor([[768, 768]], device='cuda:0'),
 'caption': ['cinematic, shoe in the streets, made from meat, photorealistic shoe, highly detailed']}

In [ ]:
c, uc = model.conditioner.get_unconditional_conditioning(
    print('crop_coords_top_left =',batch_c['crop_coords_top_left'])
    print('original_size_as_tuple =',batch_c['original_size_as_tuple'])
    print('target_size_as_tuple =',batch_c['target_size_as_tuple'])
    
    batch,
    batch_uc=batch_uc,
    force_uc_zero_embeddings=None
    if len(model.conditioner.embedders) > 0
    else [],
)

In [29]:
for k,v in c.items(): print(k,'->',type(v))

crossattn -> <class 'torch.Tensor'>
vector -> <class 'torch.Tensor'>


In [30]:
c['crossattn'].shape

torch.Size([1, 77, 2048])

In [31]:
c['vector'].shape

torch.Size([1, 2816])

In [33]:
splits = /2816
splits

tensor([0.4545, 0.1818, 0.1818, 0.1818])

In [37]:
text,orig_sz,crop,tgt_sz = torch.split(c['vector'], [1280, 512, 512, 512], dim=1)

In [38]:
text.shape,orig_sz.shape,crop.shape,tgt_sz.shape

(torch.Size([1, 1280]),
 torch.Size([1, 512]),
 torch.Size([1, 512]),
 torch.Size([1, 512]))

In [39]:
text_uc,orig_sz_uc,crop_uc,tgt_sz_uc = torch.split(uc['vector'], [1280, 512, 512, 512], dim=1)

In [40]:
text_uc.shape,orig_sz_uc.shape,crop_uc.shape,tgt_sz_uc.shape

(torch.Size([1, 1280]),
 torch.Size([1, 512]),
 torch.Size([1, 512]),
 torch.Size([1, 512]))

In [46]:
(text==text_uc).all(), (orig_sz==orig_sz_uc).all(), (crop==crop_uc).all(), (tgt_sz==tgt_sz_uc).all()

(tensor(False, device='cuda:0'),
 tensor(True, device='cuda:0'),
 tensor(True, device='cuda:0'),
 tensor(True, device='cuda:0'))

In [47]:
torch.save(c['vector'], 'c_vector')
torch.save(uc['vector'], 'uc_vector')

torch.save(c['crossattn'], 'c_crossattn')
torch.save(uc['crossattn'], 'uc_crossattn')